# 단어 표현법

## 단어의 표현 방법
단어의 표현 방법은 크게 국소 표현(Local Representation)방법과 분산 표현(Distributed Representation)방법으로 나뉨

국소 표현(=이산 표현): 해당 단어 그 자체만보고, 특정 값을 맵핑하여 단어를 표현하는 방법
EX) '강아지', '귀여운', '사랑스런' 단어가 들어오면 각각 1, 2, 3 같이 숫자로 맵핑

분산 표현(=연속 표현): 그 단어를 표현하고자 주변을 참고하여 단어를 표현하는 방법
EX) '강아지'라는 단어 주변에는 주로 '귀여운', '사랑스런'이라는 단어가 자주 등장하므로, '강아지'라는 단어는 '귀여운', '사랑스런' 느낌이다로 단어를 정의

![nn](img/word_representation01.png)

## Bag of Words(BoW)

직역하면 단어들의 가방이라는 의미
단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터 수치화 표현 방법

문서를 자동으로 분류하기 위한 방법중 하나로서, 글에 포함된 단어들의 분포를 보고 이 문서가 어떤 종류의 문서인지를 판단함
컴퓨터 비전에도 사용되었음

### 한계
**1.문맥 의미 반영 부족**
>순서를 고려하지 않기 때문에 문맥적인 의미가 무시됨

**2.희소 행렬 문제**
>문서는 대부분 서로 다른 단어로 구성되며 이에따라 문서마다 단어가 나타나지 않는 경우가 훨씬 많음
>따라서 대부분 행렬이 0으로 채워기게됨

## 문서 단어 행렬(Document-Term Matrix, DTM)

서로 다른 문서들의 BoW들을 결합한 표현 방법
행과 열을 반대로 선택하면 TDM이라고 부르기도 함
서로 다른 문서 비교 가능


### 표기법
문서 단어 행렬이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것
쉽게 생각하면 각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 생각할 수 있으며, BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어

EX)
문서1: 먹고 싶은 사과
문서2: 먹고 싶은 바나나
문서3: 길고 노란 바나나 바나나
문서4: 저는 과일이 좋아요

![nn](img/DTM_01.png)

문서 단어 행렬은 문서들을 서로 비교할 수 있도록 수치화할 수 있음
불용어를 제거한다면 더 정제된 DTM을 만들 수 있음

### 한계
**1.희소 표현**
>대부분 값이 0으로 채워질 수 있음

**2.단순 빈도 수 기반 접근**
>영어를 예시로 불용어인 'the'가 어떤 문서이든 자주 등장할 수 밖에 없음 -> 'the'의 빈도가 높다해서 문서1, 2, 3을 유사한 문서로 판단해서는 안됨

>중요한 단어에는 가중치 부여 필요

## TF-IDF(Term Frequency-Inverse Document Frequency)
모든 문서에서 자주 쓰일 수 밖에 없는 단어들(불용어)이 중요하다 인식될 수 있음
개별 문서에서 자주 등장하는 단어에 높은 가중치를 부여하되, 모든 문서에서 자주 등장하는 단어에 대해서는 패널티를 부과하는 방법

